In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
data_url = "train.csv"
data = pd.read_csv(data_url) #pandas로 csv 파일을 불러온다.

In [3]:
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [4]:
def MinMaxScaler(data):
    numerator = data - np.min(data, 0) #데이터 - 최소값. np.min(data, 0) : data의 최소값, 0차원에 대해. 열에 대한 최소값이 구해진다.
    denominator = np.max(data, 0) - np.min(data, 0) #최대값 - 최소값
    # noise term prevents the zero division
    return numerator / (denominator + 1e-7) #0으로 나눠지는 것을 막기 위해 작은 값을 더한다.

In [5]:
#의미 있는 데이터 
#Pclass : (낮은 등급(1등급)일 수록 생존률 높음
#Sex : 여성일 수록 생존률 높음
#Age : 낮을수록 생존률 높지만. 더미 있음
#Fare : 높을수록 생존률 높음 

# data["Pclass"].value_counts() #컬럼의 값들의 count
# data[(data['Pclass'] == 3) & (data["Survived"] == 1)] #조건에 맞는 행만 뽑아낸다.

#1등석 생존자 136 / 216 = 0.63
#2등석 생존자 87 / 184 = 0.47
#3등석 생존자 119 / 491 = 0.24

input_data = data[["Survived", "Pclass", "Sex", "Age", "Fare"]].dropna()
#loc등은 행을 불러올 때. 열을 불러올 때는 바로 인덱스 접근하듯이.
#dropna로 NaN 값 삭제

output_data = input_data["Survived"]
input_data = input_data.drop("Survived", axis=1) #축 지정해 줘야 한다. 축 지정하지 않은 drop은 행 삭제
input_data["Pclass"] = input_data["Pclass"].replace([1, 2, 3], [0.63, 0.47, 0.24])
input_data["Sex"] = input_data["Sex"].replace(["male", "female"], [0.19, 0.74])
input_data["Age"] = 1 - MinMaxScaler(input_data["Age"])
input_data["Fare"] = MinMaxScaler(input_data["Fare"])

x_data = input_data.values
y_data = output_data.values
y_data = np.reshape(y_data, (-1, 1))

x_data

array([[ 0.24      ,  0.19      ,  0.72882634,  0.01415106],
       [ 0.63      ,  0.74      ,  0.5277708 ,  0.13913574],
       [ 0.24      ,  0.74      ,  0.67856245,  0.01546857],
       ..., 
       [ 0.63      ,  0.74      ,  0.76652425,  0.0585561 ],
       [ 0.63      ,  0.19      ,  0.67856245,  0.0585561 ],
       [ 0.24      ,  0.19      ,  0.60316663,  0.01512699]])

In [6]:
X = tf.placeholder(tf.float32, shape=[None, 4], name="input")
Y = tf.placeholder(tf.int32, shape=[None, 1], name="labels")
keep_prob = tf.placeholder(tf.float32) #드랍아웃

# W1 = tf.Variable(tf.truncated_normal((4, 256), stddev=0.1), name="weight1")
W1 = tf.get_variable("W1", shape=[4, 256], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.zeros([256]), name="bias1")
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

# W2 = tf.Variable(tf.truncated_normal((256, 100), stddev=0.1), name="weight2")
W2 = tf.get_variable("W2", shape=[256, 512], initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.zeros([512]), name="bias2")
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

# W3 = tf.Variable(tf.truncated_normal((100, 2), stddev=0.1), name="weight3")
W3 = tf.get_variable("W3", shape=[512, 256], initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.zeros([256]), name="bias3")
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)

W4 = tf.get_variable("W4", shape=[256, 100], initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.zeros([100]), name="bias4")
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)

W5 = tf.get_variable("W5", shape=[100, 2], initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.zeros([2]), name="bias5")

Y_one_hot = tf.one_hot(Y, 2)
Y_one_hot = tf.reshape(Y_one_hot, [-1, 2])

In [10]:
logits = tf.matmul(L4, W5) + b5 #마지막 레이어는 Relu없이 그대로 내적
hypothesis = tf.nn.softmax(logits)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y_one_hot))
# optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(cost)
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

prediction = tf.argmax(hypothesis, 1) #예측된 값에서 가장 높은 값을 가지는 요소의 인덱스 추출
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1)) #정답레이블과 예상치 비교
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) #정확도 계산

In [8]:
test = pd.read_csv("test.csv")

test_data = test[["Pclass", "Sex", "Age", "Fare"]]
test_data["Pclass"] = test_data["Pclass"].replace([1, 2, 3], [0.63, 0.47, 0.24])
test_data["Sex"] = test_data["Sex"].replace(["male", "female", "NaN"], [0.19, 0.74, 30.0])
# test_data["Age"] = 1 - MinMaxScaler(input_data["Age"])
test_data["Fare"] = MinMaxScaler(input_data["Fare"])

test_data = test_data.values

print(test_data.shape)

(418, 4)


/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-cop

In [11]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer()) #초기화

    for step in range(20000):
        sess.run(optimizer, feed_dict={X: x_data, Y: y_data, keep_prob: 0.7}) #optimizer에 모든 텐서가 연결되어 있으므로 optimizer 실행
        if step % 100 == 0:
#             loss, acc = sess.run([cost, accuracy], feed_dict={X: x_data, Y: y_data})
            loss, acc = sess.run([cost, accuracy], feed_dict={X: x_data, Y: y_data, keep_prob: 0.7})
            print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(step, loss, acc))
            
    #학습 이후 테스트
#     pred = sess.run(prediction, feed_dict={X: test_data})
    pred = sess.run(prediction, feed_dict={X: test_data, keep_prob: 1})
    print("TEST")
    for p in pred: #flattend은 1차원으로 변환. [[1], [0]] -> [1, 0]. 
        print(p)


Step:     0	Loss: 0.681	Acc: 64.57%
Step:   100	Loss: 0.405	Acc: 82.35%
Step:   200	Loss: 0.402	Acc: 81.37%
Step:   300	Loss: 0.376	Acc: 83.05%
Step:   400	Loss: 0.357	Acc: 84.59%
Step:   500	Loss: 0.350	Acc: 85.01%
Step:   600	Loss: 0.339	Acc: 85.29%
Step:   700	Loss: 0.341	Acc: 85.01%
Step:   800	Loss: 0.333	Acc: 84.73%
Step:   900	Loss: 0.325	Acc: 85.57%
Step:  1000	Loss: 0.317	Acc: 86.97%
Step:  1100	Loss: 0.313	Acc: 85.99%
Step:  1200	Loss: 0.304	Acc: 86.69%
Step:  1300	Loss: 0.303	Acc: 86.83%
Step:  1400	Loss: 0.297	Acc: 87.11%
Step:  1500	Loss: 0.316	Acc: 86.41%
Step:  1600	Loss: 0.292	Acc: 87.54%
Step:  1700	Loss: 0.289	Acc: 87.68%
Step:  1800	Loss: 0.299	Acc: 87.11%
Step:  1900	Loss: 0.290	Acc: 87.68%
Step:  2000	Loss: 0.287	Acc: 88.38%
Step:  2100	Loss: 0.279	Acc: 88.52%
Step:  2200	Loss: 0.290	Acc: 87.39%
Step:  2300	Loss: 0.272	Acc: 88.66%
Step:  2400	Loss: 0.265	Acc: 88.94%
Step:  2500	Loss: 0.276	Acc: 87.54%
Step:  2600	Loss: 0.262	Acc: 89.08%
Step:  2700	Loss: 0.253	Acc: